In [3]:
fileName = 'btcusdt_1h.csv'
#USAGE set fileName
#upload appropriate file
#run all
#logs.csv contains required file

#Imports

##Import Libraries and get data

In [4]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pylab as pl
from matplotlib.pylab import date2num
from datetime import datetime, timedelta
from scipy import stats
from itertools import product
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import random
from google.colab import files

import warnings
plt.style.use('seaborn-poster')
warnings.filterwarnings("ignore")

df=pd.DataFrame(pd.read_csv('/content/'+fileName))
df['Index'] = range(0,len(df))

<ipython-input-4-40e91ff65790>:17: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


#Plotter

In [5]:
#@title Initial Values
numberOfTrades = 0
numberOfWinningTrades = 0
numberOfLosingTrades = 0
averageWinningTrade = 0.0
averageLosingTrade = 0.0
maxWinningTrade = 0.0
maxLosingTrade = 0.0
averageHoldOut = 0.0
lastPrice = 0.0
signal_column = 'signals'
initialCash = 1000
cash = initialCash
position = 0
portfolio = []
count = 0
maxDrawDown = 0.0
sharpeRatio = 0.0
sortinioRatio=0.0
priceColumn = 'close'
investmentAmount=1000
startingCapital = initialCash
commision=0.15

#Strategies

In [6]:
strategyNumber=0

##ESMA

In [7]:
length = 30
multiplier = 3
atrLength = 14
exp = True
def ESMA(data,exp, period, column="close"):
  if exp == True:
    return data[column].ewm(span=period, adjust=False).mean()
  else:
    return data[column].rolling(window=period).mean()

def ESMASignals(data,stratNumber):
  data["ma"] = ESMA(data,exp,length)
  # rangema = ATR LOGIC #### TODO ####
  data['High-Low'] = data['high'] - data['low']
  data['High-PrevClose'] = np.abs(data['high'] - data['close'].shift(1))
  data['Low-PrevClose'] = np.abs(data['low'] - data['close'].shift(1))
  data['TrueRange'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
  data["rangema"] = data['TrueRange'].rolling(window=atrLength).mean()
  data["upper"] = data["ma"] + data["rangema"]*multiplier
  data["lower"] = data["ma"] - data["rangema"] *multiplier
  data["crossUpper"] = 0
  data["crossLower"] = 0
  data["bprice"] = 0
  data["sprice"] = 0
  data["BCond"] = 0
  data["SCond"] = 0  # Same as cross upper and cross lower(unless there are nan values, none in our case)
  data["CancelBCond"] = 0
  data["CancelSCond"] = 0
  for i in range(1,len(data)):
    data.loc[i,"crossUpper"] = int(data.loc[i-1, "close"]< data.loc[i-1, "upper"] and data.loc[i,"close"]>data.loc[i,"upper"])
    data.loc[i,"crossLower"] = int(data.loc[i-1, "close"] > data.loc[i-1, "lower"] and data.loc[i,"close"]<data.loc[i,"lower"])
    data.loc[i,"bprice"] = data.loc[i,"high"] + data.loc[:i,"high"].min() if data.loc[i,"crossUpper"] == 1 else data.loc[i-1,"bprice"]
    data.loc[i,"sprice"] = data.loc[i,"low"] - data.loc[:i,"low"].min() if data.loc[i,"crossLower"] == 1 else data.loc[i-1,"bprice"]
    data.loc[i,"BCond"] = data.loc[i,"close"]>data.loc[i,"upper"]
    data.loc[i,"SCond"] = data.loc[i,"close"]<data.loc[i,"lower"]
    data.loc[i,"CancelBCond"] = data.loc[i,"BCond"] and (data.loc[i,"close"]<data.loc[i,"ma"] or data.loc[i,"high"]>=data.loc[i,"bprice"])
    data.loc[i, "CancelSCond"] = data.loc[i,"SCond"] and (data.loc[i,"close"]>data.loc[i,"ma"] or data.loc[i,"low"]<=data.loc[i,"sprice"])
  position = 0
  column = 'signals'+str(stratNumber)
  data[column] = 0
  for i in range(20,len(data)):
    if data.loc[i,"BCond"] == 1:
      if position != 1:
        data.loc[i,column] = 1
        position +=1
    elif data.loc[i,"SCond"]==1:
      if position != -1:
        data.loc[i,column] = -1
        position -= 1
    elif data.loc[i,"CancelBCond"] == 1 and position == 1:
      data.loc[i,column] = -1
      position -=1
    elif data.loc[i,"CancelSCond"] == 1 and position == -1:
      data.loc[i,column] = 1
      position +=1
    else:
      data.loc[i,column] = 0

In [8]:
ESMASignals(df,strategyNumber)
strategyNumber+=1

##CMF

In [9]:
def cmf(df, stratNumber, thresh=0):
  df_hour = df.copy()
  df_hour["datetime"] = pd.to_datetime(df_hour["datetime"])
  df_hour.set_index("datetime", inplace = True)

  df_hour.shape

  # Resampling to daily  frequency
  df_day = pd.DataFrame()
  df_day['open'] = df_hour.resample("D").open.first()
  df_day['close'] = df_hour.resample("D").close.last()
  df_day['high']=df_hour.resample("D").high.max()
  df_day['low'] = df_hour.resample("D").low.min()
  df_day['volume'] = df_hour.resample("D").volume.sum()
  df_day.reset_index(inplace=True)
  # print(df_day)

  df_hour.reset_index(inplace=True)
  signals = np.zeros(df_day.shape[0])
  MFV = df_day.volume*(2*df_day.close - df_day.low - df_day.high)/(df_day.high - df_day.low)
  res = MFV.rolling(20).sum()/df_day.volume.rolling(20).sum()
  pos = 0
  for i in range(signals.shape[0]-1):
      if res[i] > thresh and pos != 1:
          signals[i+1] = 1
          pos += 1
      elif res[i] < -thresh and pos != -1:
          signals[i+1] = -1
          pos -= 1
  df_day['signals'] = signals

  signals1 = np.zeros(df.shape[0])
  j = 0
  for i in range(df.shape[0]-1):
    if df_hour.datetime.iloc[i+1].day != df_hour.datetime.iloc[i].day:
      signals1[i] = df_day.signals[j]
      j+=1
  df['signals'+str(stratNumber)] = signals1
  return df

In [10]:
df = cmf(df,strategyNumber)
strategyNumber+=1

#COMBINER

In [11]:
######################################

In [12]:
#COMBINER CODE
signal_column = 'signals'
windowPNLs = []
windowWeights = []

In [13]:
def getCompleteInfo(df,initialCash,position=0,entryPrice=0,exitPrice=0): #assumes signal represents entry/exit, PNL= (Exit Price−Entry Price) ×  Investment amount/Entry price, returns final portfolio value
  global signal_column, priceColumn, investmentAmount
  portfolio = []
  #position = 0 #####
  cash = initialCash
  multipleBuys = False
  multipleSells = False
  #entryPrice = 0
  #exitPrice=0
  for index,row in df.iterrows():
    signal = row[signal_column]
    currPrice = row[priceColumn]
    if position==0:
      portfolio.append(cash)
      if signal==1:
        position=1
        entryPrice = currPrice
      elif signal==-1:
        position=-1
        exitPrice = currPrice
    elif position==1:
      portfolio.append(cash-investmentAmount+investmentAmount/entryPrice*currPrice)
      if signal==-1:
        change = investmentAmount/entryPrice*(currPrice-entryPrice)
        cash+=change
        cash-=commision/100*investmentAmount
        position=0
      elif signal==1:
        multipleBuys = True
    elif position==-1:
      portfolio.append(cash+investmentAmount-investmentAmount/exitPrice*currPrice)
      if signal==1:
        change = investmentAmount/exitPrice*(exitPrice-currPrice)
        cash+=change
        cash-=commision/100*investmentAmount
        position=0
      elif signal==-1:
        multipleSells = True
  if multipleBuys:
    print("Your signals attempt to hold more than 1 position")
  if multipleSells:
    print("Your signals attempt to sell more than 1 position")
  return portfolio[len(portfolio)-1]-portfolio[0]

In [14]:
#@title Main Utility Functions
def InitialiseCumSums(k):
  lst= []
  for i in range(k):
    lst.append(0)
  return lst

def InitialiseWeights(k):
  lst = []
  for i in range(k):
    lst.append(1/k)
  return lst

def GetWindow(idx,window):
  lst = []
  for i in range(idx,idx+window):
    lst.append(i)
  return lst

def GetWindowDF(df, idx, window):
  window = min(len(df)-idx,window)
  return df.iloc[GetWindow(idx,window)]

def UpdateCumSums(df,startingIndex,window,cumSums):
  global signal_column
  for w in range(window):
    for k in range(len(cumSums)):
      cumSums[k]+= df.iloc[startingIndex+w][signal_column+str(k)]
  return cumSums

def UpdateCumSumsAtIndex(df,index,cumSums,indices):
  global signal_column
  for k in range(len(cumSums)):
    cumSums[k]+=df.iloc[index][signal_column+str(indices[k])]
  return cumSums

def GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices):
  global signal_column
  PNLs = []
  minProfit = float('inf')
  prevSignalColumn = signal_column
  for i in range(numOfStrategies):
    signal_column = signal_column+str(i)
    PNLs.append(getCompleteInfo(GetWindowDF(df,startingIndex,window),currentCash,cumSums[i],entryPrices[i],exitPrices[i]))
    signal_column = prevSignalColumn
    minProfit = min(minProfit,PNLs[i])
  signal_column = prevSignalColumn
  return PNLs,minProfit

def GetPNLWeights(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())
  sum = 0
  for i in range(numOfStrategies):
    profitList[i]-=minProfit
    sum+=profitList[i]
  if sum==0:
    for i in range(numOfStrategies):
      profitList[i] = 1/numOfStrategies
  else:
    for i in range(numOfStrategies):
      profitList[i] = profitList[i]/sum
  return profitList
  #return [1,0,0,0,0]

def GetPosition(df,index,weights):
  position = 0
  for i in range(len(weights)):
    position+= weights[i]*df.iloc[index][signal_column+str(i)]
  if position>0:
    return 1
  if position<0:
    return -1
  else:
    return 0

def GetPositionByCumSums(cumSums,weights):
  position = 0
  for k in range(len(cumSums)):
    position+=weights[k]*cumSums[k]
  if position>0:
    return 1
  elif position<0:
    return -1
  else:
    return 0

def GetPositionByCumSumsThresh(cumSums,weights,thresh=0):
  position = 0
  for k in range(len(cumSums)):
    position+=weights[k]*cumSums[k]
  if position>thresh:
    return 1
  elif position<-thresh:
    return -1
  else:
    return 0

def Sign(x):
  if x>0:
    return 1
  elif x<0:
    return -1
  else:
    return 0

def GetPositionUsingMajorityCumSums(cumSums,m):
  sumOfPositions = 0
  for k in range(len(cumSums)):
    sumOfPositions+=cumSums[k]
  if abs(sumOfPositions)>=2*m-len(cumSums):
    if  sumOfPositions>0:
      return 1
    else:
      return -1
  else:
    return 0


def UpdatePricesAtIndex(df,index,cumSums,entryPrices,exitPrices):
  global priceColumn,signal_column
  price = df.iloc[index][priceColumn]
  for k in range(len(cumSums)):
    if cumSums[k]==1 and df.iloc[index][signal_column+str(k)]==1:
      entryPrices[k] = price
    elif cumSums[k]==-1 and df.iloc[index][signal_column+str(k)]==-1:
      exitPrices[k] = price
  return entryPrices,exitPrices


In [15]:
def dummyStrat():
  return 0

In [16]:
#@title Utility functions Ayman

def GetPNLWeightsLatestBestMax(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices,weights):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())

  maxProfit = -float('-inf')
  for i in range(len(profitList)):
    maxProfit = abs(max(maxProfit,profitList[i]))

  aymanFactor = 1/maxProfit

  for i in range(len(weights)):
    weights[i] = weights[i] + profitList[i]*aymanFactor

  maxWtInd = 0
  maxWt = float('-inf')
  for i in range(len(weights)):
    if(weights[i]>maxWt):
      maxWtInd = i
      maxWt = weights[i]

  res = []
  for i in range(len(weights)):
    if i==maxWtInd:
      res.append(1)
    else:
      res.append(0)
  return res

def GetPNLWeightsPrioritiseLatestMax(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices,weights):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())
  # print(windowPNLs)
  newList = []
  for i in range(numOfStrategies):
    sum = 0
    cur = 1
    for j in range(len(windowPNLs)-1,-1,-1):
      sum = sum + windowPNLs[j][i]/cur
      cur = cur+1
    newList.append(sum)

  maxWtInd = 0
  maxWt = float('-inf')
  for i in range(len(weights)):
    if(newList[i]>maxWt):
      maxWtInd = i
      maxWt = newList[i]

  res = []
  for i in range(len(weights)):
    if i==maxWtInd:
      res.append(1)
    else:
      res.append(0)
  return res

def GetPNLWeightsExpDecayComb(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices,weights):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())

  newList = []
  for i in range(numOfStrategies):
    aymanFactor = 0.8
    sum = 0
    for j in range(len(windowPNLs)-1,-1,-1):
      sum = sum + windowPNLs[j][i]*aymanFactor
      aymanFactor = aymanFactor*0.8
    newList.append(sum)

  minNum = min(newList)

  sum = 0
  for i in range(numOfStrategies):
    newList[i]-=minProfit
    sum+=newList[i]
  if sum==0:
    for i in range(numOfStrategies):
      newList[i] = 1/numOfStrategies
  else:
    for i in range(numOfStrategies):
      newList[i] = profitList[i]/sum
  return newList


def GetPNLWeightsExpDecayMax(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices,weights):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())

  newList = []
  for i in range(numOfStrategies):
    aymanFactor = 0.8
    sum = 0
    for j in range(len(windowPNLs)-1,-1,-1):
      sum = sum + windowPNLs[j][i]*aymanFactor
      aymanFactor = aymanFactor*0.8
    newList.append(sum)

  maxWtInd = 0
  maxWt = float('-inf')
  for i in range(len(weights)):
    if(newList[i]>maxWt):
      maxWtInd = i
      maxWt = newList[i]

  res = []
  for i in range(len(weights)):
    if i==maxWtInd:
      res.append(1)
    else:
      res.append(0)
  return res

def GetPNLWeightsNormal(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())
  sum = 0
  maxAbsProfit = max([abs(x) for x in profitList])
  for i in range(numOfStrategies):
      profitList[i] = profitList[i]/maxAbsProfit
  return profitList

In [17]:
#@title Window Weight Strategy
#THOUGHTS:
#Each strategy runs for window iterations, then combiner uses signals from both strategies to get new set of signals

#Bookkeeping  for stratgies - entryPrice, exitPrice, cumSum
def CombineUsingWeightedWindows(df, strategyList, window, combType='Standard', indices = []): #Assume that function takes l,r values to populate, ith strategy populates column signal+'i'
  global signal_column, priceColumn, investmentAmount, startingCapital,windowWeights
  if len(indices)==0:
    for i in range(len(strategyList)):
      indices.append(i)
  currentCash = startingCapital
  cumSums = InitialiseCumSums(len(strategyList))
  entryPrices = InitialiseCumSums(len(strategyList))
  exitPrices = InitialiseCumSums(len(strategyList))
  weights = InitialiseWeights(len(strategyList))
  signals = []
  position = 0

  for i in range(window):
    #cumSums = UpdateCumSumsAtIndex(df,i,cumSums)
    #entryPrices,exitPrices = UpdatePricesAtIndex(df,i,cumSums,entryPrices,exitPrices)
    signals.append(0)
  for i in range(window,len(df),window):
    if combType=='Standard':
      weights = GetPNLWeights(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices)
    elif combType=='ExpMax':
      weights = GetPNLWeightsExpDecayMax(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices,weights)
    elif combType=='ExpComb':
      weights = GetPNLWeightsExpDecayComb(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices,weights)
    elif combType=='Normal':
      weights = GetPNLWeightsNormal(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices)
    elif combType=='Best':
      weights = GetPNLWeightsLatestBestMax(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices,weights)
    elif combType=='Latest':
      weights = GetPNLWeightsPrioritiseLatestMax(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices,weights)

    windowWeights.append(weights.copy())
    for j in range(i-window,min(i,len(df))):
      cumSums = UpdateCumSumsAtIndex(df,j,cumSums,indices)
      entryPrices,exitPrices = UpdatePricesAtIndex(df,j,cumSums,entryPrices,exitPrices)
    cumSumsCopy = cumSums.copy()
    for w in range(min(i+window,len(df))-i):
      #cumSums = UpdateCumSumsAtIndex(df,i+w,cumSums)
      #entryPrices,exitPrices = UpdatePricesAtIndex(df,i+w,cumSums,entryPrices,exitPrices)
      newPosition = GetPositionByCumSums(cumSumsCopy,weights)
      cumSumsCopy = UpdateCumSumsAtIndex(df,i+w,cumSumsCopy,indices)
      signal = Sign(newPosition-position) #if position = 1, and newPos = -1, signal=-1. i.e. position != newPosition
      signals.append(signal)
      currentCash += -1*signal*df.iloc[i+w][priceColumn]
      position += signal
    #i+=window
  df[signal_column] = signals

In [18]:
#@title Window Weight Strategy Threshold
#THOUGHTS:
#Each strategy runs for window iterations, then combiner uses signals from both strategies to get new set of signals

#Bookkeeping  for stratgies - entryPrice, exitPrice, cumSum
def CombineUsingWeightedWindowsThresh(df, strategyList, window, threshold, indices = []): #Assume that function takes l,r values to populate, ith strategy populates column signal+'i'
  global signal_column, priceColumn, investmentAmount, startingCapital,windowWeights
  if len(indices)==0:
    for i in range(len(strategyList)):
      indices.append(i)
  currentCash = startingCapital
  cumSums = InitialiseCumSums(len(strategyList))
  entryPrices = InitialiseCumSums(len(strategyList))
  exitPrices = InitialiseCumSums(len(strategyList))
  weights = InitialiseWeights(len(strategyList))
  signals = []
  position = 0

  for i in range(window):
    #cumSums = UpdateCumSumsAtIndex(df,i,cumSums)
    #entryPrices,exitPrices = UpdatePricesAtIndex(df,i,cumSums,entryPrices,exitPrices)
    signals.append(0)
  for i in range(window,len(df),window):

    weights = GetPNLWeights(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices)
    windowWeights.append(weights.copy())
    for j in range(i-window,min(i,len(df))):
      cumSums = UpdateCumSumsAtIndex(df,j,cumSums,indices)
      entryPrices,exitPrices = UpdatePricesAtIndex(df,j,cumSums,entryPrices,exitPrices)
    cumSumsCopy = cumSums.copy()
    for w in range(min(i+window,len(df))-i):
      #cumSums = UpdateCumSumsAtIndex(df,i+w,cumSums)
      #entryPrices,exitPrices = UpdatePricesAtIndex(df,i+w,cumSums,entryPrices,exitPrices)
      newPosition = GetPositionByCumSumsThresh(cumSumsCopy,weights,threshold)
      cumSumsCopy = UpdateCumSumsAtIndex(df,i+w,cumSumsCopy,indices)
      signal = Sign(newPosition-position) #if position = 1, and newPos = -1, signal=-1. i.e. position != newPosition
      signals.append(signal)
      currentCash += -1*signal*df.iloc[i+w][priceColumn]
      position += signal
    #i+=window
  df[signal_column] = signals

In [29]:
CombineUsingWeightedWindowsThresh(df,[dummyStrat,dummyStrat],500,0.3)

In [30]:
df['Index'] = range(len(df))
df[['Index','datetime','open','high','low','close','volume','signals']].to_csv('/content/logs.csv',encoding='utf-8',index=False)